# PHÂN TÍCH DEMOGRAPHICS (Nhân khẩu học)
## Phần EDA của Bảo

---

### Mục tiêu:
1. Phân tích AGE (Độ tuổi)
2. Phân tích SEX (Giới tính)
3. Phân tích EDUCATION (Học vấn)
4. Phân tích MARRIAGE (Hôn nhân)
5. Phân tích LIMIT_BAL (Hạn mức tín dụng)
6. Tạo biểu đồ cho báo cáo

---

## 1. Import thư viện và load dữ liệu

In [ ]:
# Import thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Cài đặt style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Import thành công!")

In [ ]:
# Load dữ liệu
df = pd.read_csv('../data/raw/UCI_Credit_Card.csv')

print(f"📊 Kích thước dữ liệu: {df.shape[0]:,} hàng x {df.shape[1]} cột")
print(f"\n🎯 Tỷ lệ Default: {df['default.payment.next.month'].mean()*100:.2f}%")
print(f"🎯 Tỷ lệ No Default: {(1-df['default.payment.next.month'].mean())*100:.2f}%")

In [ ]:
# Xem 5 dòng đầu
df.head()

---
## 2. Phân tích AGE (Độ tuổi)

In [ ]:
# Thống kê mô tả
print("=" * 60)
print("THỐNG KÊ ĐỘ TUỔI")
print("=" * 60)
print(f"Tuổi trung bình: {df['AGE'].mean():.1f} tuổi")
print(f"Tuổi trung vị: {df['AGE'].median():.0f} tuổi")
print(f"Độ lệch chuẩn: {df['AGE'].std():.1f}")
print(f"Tuổi nhỏ nhất: {df['AGE'].min():.0f} tuổi")
print(f"Tuổi lớn nhất: {df['AGE'].max():.0f} tuổi")
print(f"\nPhân vị 25%: {df['AGE'].quantile(0.25):.0f} tuổi")
print(f"Phân vị 50%: {df['AGE'].quantile(0.50):.0f} tuổi")
print(f"Phân vị 75%: {df['AGE'].quantile(0.75):.0f} tuổi")
print("=" * 60)

In [ ]:
# So sánh độ tuổi giữa Default vs No Default
print("\n" + "=" * 60)
print("SO SÁNH ĐỘ TUỔI THEO DEFAULT STATUS")
print("=" * 60)

age_by_default = df.groupby('default.payment.next.month')['AGE'].describe()
print(age_by_default)

print("\n📌 Nhận xét:")
mean_no_default = df[df['default.payment.next.month']==0]['AGE'].mean()
mean_default = df[df['default.payment.next.month']==1]['AGE'].mean()

if mean_default < mean_no_default:
    print(f"- Người Default có độ tuổi trung bình THẤP HƠN ({mean_default:.1f} vs {mean_no_default:.1f})")
    print("- Người trẻ có xu hướng default nhiều hơn")
else:
    print(f"- Người Default có độ tuổi trung bình CAO HƠN ({mean_default:.1f} vs {mean_no_default:.1f})")
    print("- Người lớn tuổi có xu hướng default nhiều hơn")

In [ ]:
# Biểu đồ 1: Histogram + KDE
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Subplot 1: Histogram với KDE
sns.histplot(data=df, x='AGE', hue='default.payment.next.month', 
             kde=True, bins=30, ax=axes[0], palette=['#2ecc71', '#e74c3c'])
axes[0].set_title('Phân bố độ tuổi theo Default Status', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Độ tuổi (Age)', fontsize=12)
axes[0].set_ylabel('Số lượng (Count)', fontsize=12)
axes[0].legend(['No Default', 'Default'], title='Status')
axes[0].grid(alpha=0.3)

# Subplot 2: Box plot
sns.boxplot(data=df, x='default.payment.next.month', y='AGE', 
            ax=axes[1], palette=['#2ecc71', '#e74c3c'])
axes[1].set_title('So sánh độ tuổi: Default vs No Default', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Default Status (0=No, 1=Yes)', fontsize=12)
axes[1].set_ylabel('Độ tuổi (Age)', fontsize=12)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/age_analysis.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu biểu đồ: reports/figures/age_analysis.png")
plt.show()

---
## 3. Phân tích SEX (Giới tính)

In [ ]:
# Mapping
sex_map = {1: 'Male', 2: 'Female'}
df['SEX_NAME'] = df['SEX'].map(sex_map)

# Thống kê
print("=" * 60)
print("PHÂN BỐ GIỚI TÍNH")
print("=" * 60)
sex_counts = df['SEX_NAME'].value_counts()
sex_pct = df['SEX_NAME'].value_counts(normalize=True) * 100

for sex in sex_counts.index:
    print(f"{sex}: {sex_counts[sex]:,} người ({sex_pct[sex]:.2f}%)")
print("=" * 60)

In [ ]:
# Tỷ lệ Default theo giới tính
print("\n" + "=" * 60)
print("TỶ LỆ DEFAULT THEO GIỚI TÍNH")
print("=" * 60)

sex_default = pd.crosstab(df['SEX_NAME'], df['default.payment.next.month'], normalize='index') * 100
print(sex_default)

print("\n📌 Nhận xét:")
for sex in sex_default.index:
    default_rate = sex_default.loc[sex, 1]
    print(f"- {sex}: {default_rate:.2f}% default")

In [ ]:
# Biểu đồ 2: Count plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Subplot 1: Count plot
sns.countplot(data=df, x='SEX_NAME', hue='default.payment.next.month', 
              ax=axes[0], palette=['#2ecc71', '#e74c3c'])
axes[0].set_title('Số lượng Default theo Giới tính', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Giới tính', fontsize=12)
axes[0].set_ylabel('Số lượng', fontsize=12)
axes[0].legend(['No Default', 'Default'], title='Status')
axes[0].grid(alpha=0.3, axis='y')

# Subplot 2: Tỷ lệ % Default
sex_default_rate = df.groupby('SEX_NAME')['default.payment.next.month'].mean() * 100
sex_default_rate.plot(kind='bar', ax=axes[1], color=['#3498db', '#e91e63'])
axes[1].set_title('Tỷ lệ % Default theo Giới tính', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Giới tính', fontsize=12)
axes[1].set_ylabel('Tỷ lệ Default (%)', fontsize=12)
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].grid(alpha=0.3, axis='y')

# Thêm giá trị lên cột
for i, v in enumerate(sex_default_rate):
    axes[1].text(i, v + 0.5, f'{v:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/figures/sex_analysis.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu biểu đồ: reports/figures/sex_analysis.png")
plt.show()

---
## 4. Phân tích EDUCATION (Học vấn)

In [ ]:
# Mapping
education_map = {1: 'Graduate', 2: 'University', 3: 'High School', 4: 'Others', 0: 'Unknown', 5: 'Unknown', 6: 'Unknown'}
df['EDUCATION_NAME'] = df['EDUCATION'].map(education_map)

# Thống kê
print("=" * 60)
print("PHÂN BỐ HỌC VẤN")
print("=" * 60)
edu_counts = df['EDUCATION_NAME'].value_counts()
edu_pct = df['EDUCATION_NAME'].value_counts(normalize=True) * 100

for edu in edu_counts.index:
    print(f"{edu}: {edu_counts[edu]:,} người ({edu_pct[edu]:.2f}%)")
print("=" * 60)

In [ ]:
# Tỷ lệ Default theo học vấn
print("\n" + "=" * 60)
print("TỶ LỆ DEFAULT THEO HỌC VẤN")
print("=" * 60)

edu_default_rate = df.groupby('EDUCATION_NAME')['default.payment.next.month'].mean() * 100
edu_default_rate = edu_default_rate.sort_values(ascending=False)

for edu in edu_default_rate.index:
    print(f"{edu}: {edu_default_rate[edu]:.2f}% default")

print("\n📌 Nhận xét:")
highest_default = edu_default_rate.idxmax()
lowest_default = edu_default_rate.idxmin()
print(f"- Học vấn có tỷ lệ default CAO NHẤT: {highest_default} ({edu_default_rate[highest_default]:.2f}%)")
print(f"- Học vấn có tỷ lệ default THẤP NHẤT: {lowest_default} ({edu_default_rate[lowest_default]:.2f}%)")

In [ ]:
# Biểu đồ 3: Education analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Subplot 1: Count plot
order = df['EDUCATION_NAME'].value_counts().index
sns.countplot(data=df, x='EDUCATION_NAME', hue='default.payment.next.month', 
              order=order, ax=axes[0], palette=['#2ecc71', '#e74c3c'])
axes[0].set_title('Số lượng Default theo Học vấn', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Học vấn', fontsize=12)
axes[0].set_ylabel('Số lượng', fontsize=12)
axes[0].legend(['No Default', 'Default'], title='Status')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(alpha=0.3, axis='y')

# Subplot 2: Tỷ lệ % Default
edu_default_rate.plot(kind='barh', ax=axes[1], color='#e74c3c')
axes[1].set_title('Tỷ lệ % Default theo Học vấn', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Tỷ lệ Default (%)', fontsize=12)
axes[1].set_ylabel('Học vấn', fontsize=12)
axes[1].grid(alpha=0.3, axis='x')

# Thêm giá trị
for i, v in enumerate(edu_default_rate):
    axes[1].text(v + 0.3, i, f'{v:.1f}%', va='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/figures/education_analysis.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu biểu đồ: reports/figures/education_analysis.png")
plt.show()

---
## 5. Phân tích MARRIAGE (Tình trạng hôn nhân)

In [ ]:
# Mapping
marriage_map = {1: 'Married', 2: 'Single', 3: 'Others', 0: 'Unknown'}
df['MARRIAGE_NAME'] = df['MARRIAGE'].map(marriage_map)

# Thống kê
print("=" * 60)
print("PHÂN BỐ TÌNH TRẠNG HÔN NHÂN")
print("=" * 60)
marriage_counts = df['MARRIAGE_NAME'].value_counts()
marriage_pct = df['MARRIAGE_NAME'].value_counts(normalize=True) * 100

for status in marriage_counts.index:
    print(f"{status}: {marriage_counts[status]:,} người ({marriage_pct[status]:.2f}%)")
print("=" * 60)

In [ ]:
# Tỷ lệ Default theo hôn nhân
print("\n" + "=" * 60)
print("TỶ LỆ DEFAULT THEO TÌNH TRẠNG HÔN NHÂN")
print("=" * 60)

marriage_default_rate = df.groupby('MARRIAGE_NAME')['default.payment.next.month'].mean() * 100
marriage_default_rate = marriage_default_rate.sort_values(ascending=False)

for status in marriage_default_rate.index:
    print(f"{status}: {marriage_default_rate[status]:.2f}% default")

In [ ]:
# Biểu đồ 4: Marriage analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Subplot 1: Count plot
order = df['MARRIAGE_NAME'].value_counts().index
sns.countplot(data=df, x='MARRIAGE_NAME', hue='default.payment.next.month', 
              order=order, ax=axes[0], palette=['#2ecc71', '#e74c3c'])
axes[0].set_title('Số lượng Default theo Tình trạng hôn nhân', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Tình trạng hôn nhân', fontsize=12)
axes[0].set_ylabel('Số lượng', fontsize=12)
axes[0].legend(['No Default', 'Default'], title='Status')
axes[0].grid(alpha=0.3, axis='y')

# Subplot 2: Pie chart
marriage_counts.plot(kind='pie', ax=axes[1], autopct='%1.1f%%', 
                     colors=['#3498db', '#e91e63', '#9b59b6'], startangle=90)
axes[1].set_title('Phân bố Tình trạng hôn nhân', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.savefig('../reports/figures/marriage_analysis.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu biểu đồ: reports/figures/marriage_analysis.png")
plt.show()

---
## 6. Phân tích LIMIT_BAL (Hạn mức tín dụng)

In [ ]:
# Thống kê
print("=" * 60)
print("THỐNG KÊ HẠN MỨC TÍN DỤNG (NT$)")
print("=" * 60)
print(f"Hạn mức trung bình: {df['LIMIT_BAL'].mean():,.0f} NT$")
print(f"Hạn mức trung vị: {df['LIMIT_BAL'].median():,.0f} NT$")
print(f"Độ lệch chuẩn: {df['LIMIT_BAL'].std():,.0f}")
print(f"Hạn mức nhỏ nhất: {df['LIMIT_BAL'].min():,.0f} NT$")
print(f"Hạn mức lớn nhất: {df['LIMIT_BAL'].max():,.0f} NT$")
print(f"\nPhân vị 25%: {df['LIMIT_BAL'].quantile(0.25):,.0f} NT$")
print(f"Phân vị 50%: {df['LIMIT_BAL'].quantile(0.50):,.0f} NT$")
print(f"Phân vị 75%: {df['LIMIT_BAL'].quantile(0.75):,.0f} NT$")
print("=" * 60)

In [ ]:
# So sánh hạn mức theo Default
print("\n" + "=" * 60)
print("SO SÁNH HẠN MỨC THEO DEFAULT STATUS")
print("=" * 60)

limit_by_default = df.groupby('default.payment.next.month')['LIMIT_BAL'].describe()
print(limit_by_default)

print("\n📌 Nhận xét:")
mean_no_default = df[df['default.payment.next.month']==0]['LIMIT_BAL'].mean()
mean_default = df[df['default.payment.next.month']==1]['LIMIT_BAL'].mean()

if mean_default < mean_no_default:
    diff = mean_no_default - mean_default
    print(f"- Người Default có hạn mức THẤP HƠN {diff:,.0f} NT$ ({mean_default:,.0f} vs {mean_no_default:,.0f})")
    print("- Hạn mức thấp → Rủi ro default cao hơn")
else:
    diff = mean_default - mean_no_default
    print(f"- Người Default có hạn mức CAO HƠN {diff:,.0f} NT$ ({mean_default:,.0f} vs {mean_no_default:,.0f})")

In [ ]:
# Biểu đồ 5: Credit Limit analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Subplot 1: Histogram
sns.histplot(data=df, x='LIMIT_BAL', hue='default.payment.next.month', 
             kde=True, bins=50, ax=axes[0], palette=['#2ecc71', '#e74c3c'])
axes[0].set_title('Phân bố Hạn mức tín dụng theo Default Status', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Hạn mức (NT$)', fontsize=12)
axes[0].set_ylabel('Số lượng', fontsize=12)
axes[0].legend(['No Default', 'Default'], title='Status')
axes[0].grid(alpha=0.3)

# Subplot 2: Box plot
sns.boxplot(data=df, x='default.payment.next.month', y='LIMIT_BAL', 
            ax=axes[1], palette=['#2ecc71', '#e74c3c'])
axes[1].set_title('So sánh Hạn mức: Default vs No Default', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Default Status (0=No, 1=Yes)', fontsize=12)
axes[1].set_ylabel('Hạn mức (NT$)', fontsize=12)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/limit_analysis.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu biểu đồ: reports/figures/limit_analysis.png")
plt.show()

---
## 7. Tổng hợp và Kết luận

In [ ]:
# Tạo bảng tổng hợp
print("=" * 80)
print("BẢNG TỔNG HỢP PHÂN TÍCH DEMOGRAPHICS")
print("=" * 80)

summary = pd.DataFrame({
    'Feature': ['AGE', 'SEX (Male)', 'SEX (Female)', 'EDUCATION (Graduate)', 
                'EDUCATION (University)', 'MARRIAGE (Married)', 'MARRIAGE (Single)', 'LIMIT_BAL'],
    'Mean/Count': [
        f"{df['AGE'].mean():.1f} tuổi",
        f"{(df['SEX']==1).sum():,} người",
        f"{(df['SEX']==2).sum():,} người",
        f"{(df['EDUCATION']==1).sum():,} người",
        f"{(df['EDUCATION']==2).sum():,} người",
        f"{(df['MARRIAGE']==1).sum():,} người",
        f"{(df['MARRIAGE']==2).sum():,} người",
        f"{df['LIMIT_BAL'].mean():,.0f} NT$"
    ],
    'Default Rate': [
        f"{df.groupby('AGE')['default.payment.next.month'].mean().mean()*100:.2f}%",
        f"{df[df['SEX']==1]['default.payment.next.month'].mean()*100:.2f}%",
        f"{df[df['SEX']==2]['default.payment.next.month'].mean()*100:.2f}%",
        f"{df[df['EDUCATION']==1]['default.payment.next.month'].mean()*100:.2f}%",
        f"{df[df['EDUCATION']==2]['default.payment.next.month'].mean()*100:.2f}%",
        f"{df[df['MARRIAGE']==1]['default.payment.next.month'].mean()*100:.2f}%",
        f"{df[df['MARRIAGE']==2]['default.payment.next.month'].mean()*100:.2f}%",
        "N/A"
    ]
})

print(summary.to_string(index=False))
print("=" * 80)

In [ ]:
# Kết luận
print("\n" + "=" * 80)
print("KẾT LUẬN PHÂN TÍCH DEMOGRAPHICS")
print("=" * 80)
print("""
1. ĐỘ TUỔI (AGE):
   - Độ tuổi trung bình: 35.5 tuổi
   - Phạm vi: 21-79 tuổi
   - Người trẻ (25-35 tuổi) có xu hướng default cao hơn

2. GIỚI TÍNH (SEX):
   - Nữ chiếm đa số (60.4%)
   - Tỷ lệ default giữa nam và nữ tương đương nhau

3. HỌC VẤN (EDUCATION):
   - Đa số có trình độ Đại học (46.8%)
   - Học vấn cao hơn → Tỷ lệ default thấp hơn

4. TÌNH TRẠNG HÔN NHÂN (MARRIAGE):
   - Single chiếm 53.2%
   - Married chiếm 45.5%
   - Single có xu hướng default cao hơn Married

5. HẠN MỨC TÍN DỤNG (LIMIT_BAL):
   - Hạn mức trung bình: 167,484 NT$
   - Người có hạn mức thấp → Rủi ro default cao hơn
   - Phạm vi rộng: 10,000 - 1,000,000 NT$

📌 NHẬN XÉT CHUNG:
- Dataset có sự phân bố tương đối cân bằng về demographics
- Các yếu tố demographics có ảnh hưởng đến khả năng default
- Cần kết hợp với financial features để dự đoán chính xác hơn
""")
print("=" * 80)

In [ ]:
# Liệt kê các biểu đồ đã tạo
print("\n✅ ĐÃ TẠO CÁC BIỂU ĐỒ SAU:")
print("1. reports/figures/age_analysis.png")
print("2. reports/figures/sex_analysis.png")
print("3. reports/figures/education_analysis.png")
print("4. reports/figures/marriage_analysis.png")
print("5. reports/figures/limit_analysis.png")
print("\n📝 Các biểu đồ này có thể dùng cho báo cáo!")